In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [7]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer

# Load the data
train = pd.read_csv('/kaggle/input/icr-identify-age-related-conditions/train.csv')
greeks = pd.read_csv('/kaggle/input/icr-identify-age-related-conditions/greeks.csv')


# Merge train and greeks based on 'Id'
merged_train = pd.merge(train, greeks, on='Id', how='left')
summary_stats = merged_train.describe()

print(summary_stats)

# # Print the number of missing values in each column
# print(merged_train.isnull().sum())





               AB            AF           AH          AM          AR  \
count  617.000000    617.000000   617.000000  617.000000  617.000000   
mean     0.477149   3502.013221   118.624513   38.968552   10.128242   
std      0.468388   2300.322717   127.838950   69.728226   10.518877   
min      0.081187    192.593280    85.200147    3.177522    8.138688   
25%      0.252107   2197.345480    85.200147   12.270314    8.138688   
50%      0.354659   3120.318960    85.200147   20.533110    8.138688   
75%      0.559763   4361.637390   113.739540   39.139886    8.138688   
max      6.161666  28688.187660  1910.123198  630.518230  178.943634   

               AX          AY          AZ           BC           BD   ...  \
count  617.000000  617.000000  617.000000   617.000000    617.000000  ...   
mean     5.545576    0.060320   10.566447     8.053012   5350.388655  ...   
std      2.551696    0.416817    4.350645    65.166943   3021.326641  ...   
min      0.699861    0.025578    3.396778  

In [4]:
# If there are missing values, fill them with a suitable method. Here we fill with the most frequent value in each column.
imputer = SimpleImputer(strategy='most_frequent')
merged_train_imputed = pd.DataFrame(imputer.fit_transform(merged_train), columns=merged_train.columns)

# One-hot encode the categorical features
categorical_cols = ['Alpha', 'Beta', 'Gamma', 'Delta', 'Epsilon']  # Please specify the categorical columns

encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
train_encoded = pd.DataFrame(encoder.fit_transform(merged_train_imputed[categorical_cols]))

# Rename the encoded columns
train_encoded.columns = encoder.get_feature_names_out(categorical_cols)

# Replace the original columns with the new one-hot encoded columns
merged_train_imputed = merged_train_imputed.drop(categorical_cols, axis=1)
merged_train_encoded = pd.concat([merged_train_imputed, train_encoded], axis=1)

# Separate the target variable and explanatory variables
X = merged_train_encoded.drop(['Id', 'Class'], axis=1)
y = merged_train_encoded['Class']

(617, 58)
